In [3]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split 
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

##### Загружаем датасет

In [4]:
df = pd.read_csv('data/insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


##### Проверяем на наличие пропусков и типы данные 

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


##### Разделение признаков и целевой переменной

In [6]:
x = df.drop(['charges'], axis=1)
y = df['charges'].values

##### Преобразование категориальных признаков в числовые значения и нормализация числовых признаков

In [7]:
x['sex'] = x['sex'].map({'male': 0, 'female': 1})
x['smoker'] = x['smoker'].map({'yes': 1, 'no': 0})
x['region'] = x['region'].map({'southwest': 0, 'southeast': 1, 'northwest': 2, 'northeast': 3})

for feature in ['age', 'bmi', 'children', 'region']:
    x[feature] = (x[feature] - x[feature].mean()) / x[feature].std()
    
x.head()

,age,sex,bmi,children,smoker,region
0,-1.438227,1,-0.453151,-0.908274,1,-1.343402
1,-1.509401,0,0.509431,-0.078738,0,-0.438331
2,-0.797655,0,0.383164,1.580335,0,-0.438331
3,-0.441782,0,-1.305043,-0.908274,0,0.466741
4,-0.512957,0,-0.292447,-0.908274,0,0.466741


##### Выполняем снижение размерности на основе МГК (мин уровень дисперсии 0.95)

In [8]:
pca = PCA(n_components=0.98) # при 0.95 растет полиномиальная линейная модель 
x = pca.fit_transform(x)
x_base = x

##### Выполняем расширение базовых признаков за счет применения полиномиальной модели (степень = 3)

In [9]:
poly = PolynomialFeatures(degree=3, include_bias=False)
x_poly = poly.fit_transform(x_base)

##### Разбиваем датасеты (базовый и расширенный) на 2 множества (обучающее и тестовое).

In [10]:
# Разделение базового набора данных на тренировочные и тестовые
x_base_train, x_base_test, y_base_train, y_base_test = train_test_split(
    x_base, y, test_size=0.3, shuffle=True
)

# Разделение расширенного набора данных на тренировочные и тестовые
x_poly_train, x_poly_test, y_poly_train, y_poly_test = train_test_split(
    x_poly, y, test_size=0.3, shuffle=True
)

##### Выполняем обучение линейной и полиномиальной моделей аналитическим методом - не используя готовое библиотечное решение.


In [11]:
def analytical_solution(x, y):
    x = np.c_[np.ones(x.shape[0]), x] # Добавляем столбец единиц для учета интерсепта
    theta = np.linalg.inv(x.T @ x) @ x.T @ y
    return theta

In [12]:
# Обучение базовой модели
linear_theta_base = analytical_solution(x_base_train, y_base_train)
print("Коэффициенты базовой модели:", linear_theta_base.flatten())

# Обучение расширенной модели
linear_theta_poly = analytical_solution(x_poly_train, y_poly_train)
print("Коэффициенты расширенной модели:", linear_theta_poly.flatten())

Коэффициенты базовой модели: [13319.85003496  2748.48674202  2425.31337545 -1956.59364206
   298.62127293 -3883.21683185 23202.41372127]
Коэффициенты расширенной модели: [ 1.74985175e+17 -9.94054747e+16  1.40686082e+16 -8.67120238e+14
  1.28007082e+14  5.84532699e+14  3.77280920e+16  7.00917709e+14
  1.39842998e+15 -3.11848603e+15 -3.73409763e+15  6.91338244e+16
 -2.92260221e+17  8.60024343e+13 -9.84552167e+13 -7.06061877e+12
 -2.56465746e+15 -2.58318140e+16 -2.42949903e+14 -7.92140393e+14
  2.03094095e+16 -9.06982597e+15 -8.60868441e+14  4.69595028e+16
  6.70161464e+15 -6.41154925e+17 -1.96974624e+17 -7.95914340e+16
  1.90568937e+13 -1.91309307e+13 -1.20205451e+12  4.26931541e+13
 -1.48207317e+15 -3.14522254e+15  2.68865330e+13 -8.72460383e+13
 -1.65037430e+14  4.02969586e+15 -3.78077314e+15  5.54325661e+13
  1.55666448e+14 -3.19465263e+15  9.25178644e+15  1.37478204e+14
 -6.38598399e+15  9.37121735e+15  7.79794860e+16 -1.45981572e+17
  4.95779509e+17 -2.20731202e+12  1.06060324e+13  

##### Выполняем обучение линейной и полиномиальной моделей градиентным спуском - не используя готовое библиотечное решение.


In [13]:
def gradient_descent(x, y, learning_rate=0.01, iterations=1000):
    m, n = x.shape
    x = np.c_[np.ones(m), x]  # Добавляем столбец единиц для интерсепта
    theta = np.zeros((n + 1,))  # Инициализация коэффициентов
    for i in range(iterations):
        predictions = x @ theta
        error = predictions - y
        theta -= (learning_rate / m) * (x.T @ error)
    return theta

In [14]:
# Обучение базовой модели
gradient_theta_base = gradient_descent(x_base_train, y_base_train, learning_rate=0.0001, iterations=100000)
print("Коэффициенты базовой модели:", gradient_theta_base.flatten())

# Обучение расширенной модели
gradient_theta_poly = gradient_descent(x_poly_train, y_poly_train, learning_rate=0.0001, iterations=100000)
print("Коэффициенты расширенной модели:", gradient_theta_poly.flatten())

Коэффициенты базовой модели: [13315.93818683  2738.45826243  2396.57970104 -1947.79704065
   333.53433322 -3570.86674983 18484.07525069]
Коэффициенты расширенной модели: [ 8727.08391416  1638.0818538   1747.45439762 -1096.36306807
   284.0825199  -1199.8295814   6843.86239255   425.36892819
   123.46666487  -198.86111033  -819.47070658 -1060.43771728
  3751.52804669   842.46262881  -140.23292929   228.19624785
   264.17713599 -1535.88991636   938.0389734    574.41874522
   365.69688695 -1529.7102293    843.96803679  -734.08585002
  3858.39020623  2625.71711827 -1584.36262495  5016.1511498
    28.60065854   341.93129198   -26.02363819  -166.77007211
  -479.50746513  2226.57216719   739.66652038  -163.07228703
  -544.21186875   -13.47241988   -38.4638657   -148.37698678
   227.4304046   -528.99886118   863.47611808  -355.08882829
   -13.97872232 -1304.80510045   736.70385791 -1214.67431872
  2209.41146962    65.26270122  -460.2204366   -402.0283913
  -803.18713903  2976.09100954  -125.44

In [15]:
def predict(x, theta):
    x = np.c_[np.ones(x.shape[0]), x]  # Добавляем столбец единиц для интерсепта, если его нет
    return x @ theta

# Предсказания для базовой модели
y_base_pred_linear = predict(x_base_test, linear_theta_base)
y_base_pred_gradient = predict(x_base_test, gradient_theta_base)

# Предсказания для полиномиальной модели
y_poly_pred_linear = predict(x_poly_test, linear_theta_poly)
y_poly_pred_gradient = predict(x_poly_test, gradient_theta_poly)

print("Метрики для базовой модели (Linear):")
print(f"MAE : {mean_absolute_error(y_base_test, y_base_pred_linear):.3f}")
print(f"MSE : {mean_squared_error(y_base_test, y_base_pred_linear):.3f}")
print(f"R²  : {r2_score(y_base_test, y_base_pred_linear):.3f}")

print("\nМетрики для полиномиальной модели (Linear):")
print(f"MAE : {mean_absolute_error(y_poly_test, y_poly_pred_linear):.3f}")
print(f"MSE : {mean_squared_error(y_poly_test, y_poly_pred_linear):.3f}")
print(f"R²  : {r2_score(y_poly_test, y_poly_pred_linear):.3f}")

print("\nМетрики для базовой модели (Gradient):")
print(f"MAE : {mean_absolute_error(y_base_test, y_base_pred_gradient):.3f}")
print(f"MSE : {mean_squared_error(y_base_test, y_base_pred_gradient):.3f}")
print(f"R²  : {r2_score(y_base_test, y_base_pred_gradient):.3f}")

print("\nМетрики для полиномиальной модели (Gradient):")
print(f"MAE : {mean_absolute_error(y_poly_test, y_poly_pred_gradient):.3f}")
print(f"MSE : {mean_squared_error(y_poly_test, y_poly_pred_gradient):.3f}")
print(f"R²  : {r2_score(y_poly_test, y_poly_pred_gradient):.3f}")

Метрики для базовой модели (Linear):
MAE : 4089.489
MSE : 32931231.001
R²  : 0.779

Метрики для полиномиальной модели (Linear):
MAE : 40129.834
MSE : 1895931374.389
R²  : -12.157

Метрики для базовой модели (Gradient):
MAE : 4621.165
MSE : 38170293.965
R²  : 0.744

Метрики для полиномиальной модели (Gradient):
MAE : 3371.608
MSE : 29768337.626
R²  : 0.793
